In [2]:
#Load libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [3]:
# Load dataset
data = pd.read_csv("Bank_Transaction_Fraud_Detection.csv")
# Display all the columns
pd.options.display.max_columns = None
data.head()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,Transaction_Time,Transaction_Amount,Merchant_ID,Transaction_Type,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,23-01-2025,16:04:07,32415.45,214e03c5-5c34-40d1-a66c-f440aa2bbd02,Transfer,Restaurant,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",POS,0,INR,+9198579XXXXXX,Bitcoin transaction,oshaXXXXX@XXXXX.com
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,11-01-2025,17:14:53,43622.60,f9e3f11f-28d3-4199-b0ca-f225a155ede6,Bill Payment,Restaurant,74622.66,POS Mobile Device,"Nashik, Maharashtra",Desktop,0,INR,+9191074XXXXXX,Grocery delivery,hredhaanXXXX@XXXXXX.com
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,25-01-2025,03:09:52,63062.56,97977d83-5486-4510-af1c-8dada3e1cfa0,Bill Payment,Groceries,66817.99,ATM,"Bhagalpur, Bihar",Desktop,0,INR,+9197745XXXXXX,Mutual fund investment,ekaniXXX@XXXXXX.com
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,19-01-2025,12:27:02,14000.72,f45cd6b3-5092-44d0-8afb-490894605184,Debit,Entertainment,58177.08,POS Mobile App,"Chennai, Tamil Nadu",Mobile,0,INR,+9195889XXXXXX,Food delivery,yaminiXXXXX@XXXXXXX.com
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,30-01-2025,18:30:46,18335.16,70dd77dd-3b00-4b2c-8ebc-cfb8af5f6741,Transfer,Entertainment,16108.56,Virtual Card,"Amritsar, Punjab",Mobile,0,INR,+9195316XXXXXX,Debt repayment,kritikaXXXX@XXXXXX.com


In [4]:
# Check the last five rows
data.tail()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,Transaction_Time,Transaction_Amount,Merchant_ID,Transaction_Type,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
199995,b8bdae19-296f-48b7-9104-e055d33a09ac,Vedhika Magar,Female,55,Dadra and Nagar Haveli and Daman and Diu,Diu,Diu Branch,Business,8d856bc7-4666-4509-a067-48d67500694a,08-01-2025,18:42:09,98513.74,72817d4a-830b-4d16-bf74-244dccfe4cc4,Credit,Restaurant,37475.11,Desktop/Laptop,"Diu, Dadra and Nagar Haveli and Daman and Diu",ATM,0,INR,+9192629XXXXXX,ATM withdrawal,vedhikaXXXXX@XXXXXXX.com
199996,635bc099-8a93-48ee-829a-bf2283fe8fda,Aashi Pai,Male,51,Manipur,Kangpokpi,Kangpokpi Branch,Business,f2890dbd-4e01-445d-97f5-ac56886e9037,01-01-2025,20:51:21,40593.55,087718f9-1faa-44ef-b162-24d20ddc903c,Withdrawal,Groceries,53037.20,ATM,"Kangpokpi, Manipur",ATM,0,INR,+9198116XXXXXX,Subscription box,aashiXXX@XXXXXX.com
199997,c1b31cc2-0905-47e8-8cc5-6461d1f3ba33,Dayita Shan,Female,41,Chandigarh,Chandigarh,Chandigarh Branch,Savings,f714b758-7539-474d-b676-5fa7e2480141,28-01-2025,10:47:40,61579.70,a313cfbc-ef3d-4e59-8347-a948ac292a6f,Withdrawal,Health,96225.36,ATM,"Chandigarh, Chandigarh",Desktop,0,INR,+9192601XXXXXX,Charity donation,dayitaXXXX@XXXXXXX.com
199998,32bb8e66-f3fa-43bf-8242-dab9a6116310,Unnati Vyas,Female,28,Telangana,Nizamabad,Nizamabad Branch,Checking,f6903b6a-b582-47ea-95d5-aff16bdec950,08-01-2025,06:26:41,39488.22,08ef8813-dea0-42bf-9df5-0a63fe07673d,Debit,Electronics,89599.90,Voice Assistant,"Nizamabad, Telangana",Desktop,0,INR,+9197537XXXXXX,Tourist attraction payment,unnatiXXX@XXXXX.com
199999,f3dd92ef-b17a-45d2-b6cb-fdbee20843fe,Gopal Rout,Male,34,Nagaland,Kohima,Kohima Branch,Business,b44b6e8a-1036-4ec1-b492-5e7ffc7baf6d,08-01-2025,15:26:19,58622.49,e0d4aa67-43c0-4aed-836b-698cfaf2df41,Debit,Electronics,15066.24,Virtual Card,"Kohima, Nagaland",Mobile,0,INR,+9193961XXXXXX,Installment payment,gopalXXXXX@XXXXXXX.com


In [5]:
# Check the number of rows and columns in the dataset
data.shape

(200000, 24)

In [6]:
# Get the information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Customer_ID              200000 non-null  object 
 1   Customer_Name            200000 non-null  object 
 2   Gender                   200000 non-null  object 
 3   Age                      200000 non-null  int64  
 4   State                    200000 non-null  object 
 5   City                     200000 non-null  object 
 6   Bank_Branch              200000 non-null  object 
 7   Account_Type             200000 non-null  object 
 8   Transaction_ID           200000 non-null  object 
 9   Transaction_Date         200000 non-null  object 
 10  Transaction_Time         200000 non-null  object 
 11  Transaction_Amount       200000 non-null  float64
 12  Merchant_ID              200000 non-null  object 
 13  Transaction_Type         200000 non-null  object 
 14  Merc

In [7]:
# Checking the Null values in the dataset
data.isnull().sum()

Customer_ID                0
Customer_Name              0
Gender                     0
Age                        0
State                      0
City                       0
Bank_Branch                0
Account_Type               0
Transaction_ID             0
Transaction_Date           0
Transaction_Time           0
Transaction_Amount         0
Merchant_ID                0
Transaction_Type           0
Merchant_Category          0
Account_Balance            0
Transaction_Device         0
Transaction_Location       0
Device_Type                0
Is_Fraud                   0
Transaction_Currency       0
Customer_Contact           0
Transaction_Description    0
Customer_Email             0
dtype: int64

In [8]:
# Check duplication in the dataset
data.duplicated().any()

np.False_

### Identify Categorical Features

In [9]:
# Select only categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
print("Categorical Features:", categorical_columns)

Categorical Features: Index(['Customer_ID', 'Customer_Name', 'Gender', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Merchant_ID', 'Transaction_Type',
       'Merchant_Category', 'Transaction_Device', 'Transaction_Location',
       'Device_Type', 'Transaction_Currency', 'Customer_Contact',
       'Transaction_Description', 'Customer_Email'],
      dtype='object')


In [10]:
# Convert date column to datetime format
data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])
data['Transaction_Time'] = pd.to_datetime(data['Transaction_Time'], format='%H:%M:%S')

# Extract useful numerical features
data['Transaction_Hour'] = data['Transaction_Time'].dt.hour
data['Transaction_Day'] = data['Transaction_Date'].dt.day
data['Transaction_Month'] = data['Transaction_Date'].dt.month
data['Transaction_Weekday'] = data['Transaction_Date'].dt.weekday

# Drop original date columns
data.drop(['Transaction_Date', 'Transaction_Time'], axis=1, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_7052\3607677854.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])


### Label Encoding for Binary Categories

In [11]:
# Define binary categorical columns
binary_columns = ['Gender', 'Account_Type']

# Apply Label Encoding
label_encoder = LabelEncoder()
for col in binary_columns:
    data[col] = label_encoder.fit_transform(data[col])

# Check the result
print(data[binary_columns].head())


   Gender  Account_Type
0       1             2
1       0             0
2       1             2
3       0             0
4       0             2


### One-Hot Encoding for Multi-Class Categories

In [12]:

# One-Hot Encoding (For Low-Cardinality Columns)
low_cardinality_cols = ['State', 'City', 'Bank_Branch', 'Transaction_Type', 
                        'Merchant_Category', 'Transaction_Device', 'Device_Type', 
                        'Transaction_Currency']
data = pd.get_dummies(data, columns=low_cardinality_cols, drop_first=True)

# Frequency Encoding (For High-Cardinality Columns)
high_cardinality_cols = ['Customer_Email', 'Merchant_ID']
for col in high_cardinality_cols:
    data[col] = data[col].map(data[col].value_counts(normalize=True))

# Hash Encoding (For Transaction_Description)
hasher = FeatureHasher(n_features=50, input_type="string")
hashed_features = hasher.transform(data['Transaction_Description'].apply(lambda x: x.split()))
hashed_df = pd.DataFrame(hashed_features.toarray())

# Drop original text column and add hashed features
data = pd.concat([data.drop(columns=['Transaction_Description']), hashed_df], axis=1)

# Check new dataset shape
print("New shape after encoding:", data.shape)

New shape after encoding: (200000, 419)


In [13]:
data.shape

(200000, 419)

In [14]:
data.head()

,Customer_ID,Customer_Name,Gender,Age,Account_Type,Transaction_ID,Transaction_Amount,Merchant_ID,Account_Balance,Transaction_Location,Is_Fraud,Customer_Contact,Customer_Email,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Weekday,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_

In [15]:
# Replacing True / False with 0/1
data = data.replace({True: 1, False: 0}).infer_objects(copy=False)

C:\Users\User\AppData\Local\Temp\ipykernel_7052\3729861657.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0}).infer_objects(copy=False)


In [16]:
# Rename hashed columns properly
hashed_columns = [f"hash_{i}" for i in range(50)]  # Adjust 50 if needed
data.rename(columns=dict(zip(range(50), hashed_columns)), inplace=True)

# Print final dataset info
print("Final dataset shape:", data.shape)
print("Column names:", data.columns[:20])  # Show first 20 column names


Final dataset shape: (200000, 419)
Column names: Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'Account_Type',
       'Transaction_ID', 'Transaction_Amount', 'Merchant_ID',
       'Account_Balance', 'Transaction_Location', 'Is_Fraud',
       'Customer_Contact', 'Customer_Email', 'Transaction_Hour',
       'Transaction_Day', 'Transaction_Month', 'Transaction_Weekday',
       'State_Andhra Pradesh', 'State_Arunachal Pradesh', 'State_Assam'],
      dtype='object')


In [17]:
# Displaying the first five rows
data.head()

,Customer_ID,Customer_Name,Gender,Age,Account_Type,Transaction_ID,Transaction_Amount,Merchant_ID,Account_Balance,Transaction_Location,Is_Fraud,Customer_Contact,Customer_Email,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Weekday,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_

In [18]:
# Encode cyclical features using sine and cosine transformations
data['Transaction_Hour_Sin'] = np.sin(2 * np.pi * data['Transaction_Hour'] / 24)
data['Transaction_Hour_Cos'] = np.cos(2 * np.pi * data['Transaction_Hour'] / 24)

data['Transaction_Day_Sin'] = np.sin(2 * np.pi * data['Transaction_Day'] / 31)
data['Transaction_Day_Cos'] = np.cos(2 * np.pi * data['Transaction_Day'] / 31)

data['Transaction_Month_Sin'] = np.sin(2 * np.pi * data['Transaction_Month'] / 12)
data['Transaction_Month_Cos'] = np.cos(2 * np.pi * data['Transaction_Month'] / 12)

data['Transaction_Weekday_Sin'] = np.sin(2 * np.pi * data['Transaction_Weekday'] / 7)
data['Transaction_Weekday_Cos'] = np.cos(2 * np.pi * data['Transaction_Weekday'] / 7)

# Drop the original columns after transformation
data.drop(columns=['Transaction_Hour', 'Transaction_Day', 'Transaction_Month', 'Transaction_Weekday'], inplace=True)


### Feature Engineering

In [19]:
# Transaction Velocity Features: Count transactions per user in different time windows
data['Transaction_1hr'] = data.groupby('Customer_ID')['Transaction_Amount'].rolling(window=1).count().reset_index(0, drop=True)
data['Transaction_24hr'] = data.groupby('Customer_ID')['Transaction_Amount'].rolling(window=24).count().reset_index(0, drop=True)
data['Transaction_7days'] = data.groupby('Customer_ID')['Transaction_Amount'].rolling(window=7).count().reset_index(0, drop=True)

# Average Transaction Amount per User in Different Time Windows
data['Avg_Transaction_24hr'] = data.groupby('Customer_ID')['Transaction_Amount'].rolling(window=24).mean().reset_index(0, drop=True)
data['Avg_Transaction_7days'] = data.groupby('Customer_ID')['Transaction_Amount'].rolling(window=7).mean().reset_index(0, drop=True)

# Device Consistency: Flag if transaction is from a new device
data['New_Device_Used'] = data.groupby('Customer_ID')['Transaction_Location'].transform(lambda x: x != x.shift()).astype(int)

# Merchant Patterns: Flag if transaction is made to an unusual merchant for the user
data['Unusual_Merchant'] = data.groupby('Customer_ID')['Merchant_ID'].transform(lambda x: x != x.mode()[0]).astype(int)

# Behavioral Patterns: Compare Transaction_Amount with historical transactions
data['Transaction_Deviation'] = data.groupby('Customer_ID')['Transaction_Amount'].transform(lambda x: (x - x.mean()) / x.std())

# Fill NaN values in newly created features
data.fillna(0, inplace=True)





In [20]:
# Display the first few rows to check the new features
data.head()

,Customer_ID,Customer_Name,Gender,Age,Account_Type,Transaction_ID,Transaction_Amount,Merchant_ID,Account_Balance,Transaction_Location,Is_Fraud,Customer_Contact,Customer_Email,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_Branch_Hazaribagh Branch,Bank_Branch_Hisar Branch,Bank_Branch_Howrah Br

### Feature Scaling

In [21]:
# Select numerical features to scale
num_features = ['Transaction_Amount', 'Account_Balance', 'Transaction_1hr', 
                'Transaction_24hr', 'Transaction_7days', 'Avg_Transaction_24hr', 
                'Avg_Transaction_7days', 'Transaction_Deviation', 'Age']

# Initialize the scaler
scaler = StandardScaler()

# Apply scaling
data[num_features] = scaler.fit_transform(data[num_features])

In [22]:
# View data
data.head()

,Customer_ID,Customer_Name,Gender,Age,Account_Type,Transaction_ID,Transaction_Amount,Merchant_ID,Account_Balance,Transaction_Location,Is_Fraud,Customer_Contact,Customer_Email,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_Branch_Hazaribagh Branch,Bank_Branch_Hisar Branch,Bank_Branch_Howrah Br

In [23]:
# View the last five rows of data
data.tail()

,Customer_ID,Customer_Name,Gender,Age,Account_Type,Transaction_ID,Transaction_Amount,Merchant_ID,Account_Balance,Transaction_Location,Is_Fraud,Customer_Contact,Customer_Email,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_Branch_Hazaribagh Branch,Bank_Branch_Hisar Branch,Bank_Branch_Howrah Br

In [24]:
# Drop non-numeric columns
columns_to_drop = ['Merchant_ID', 'Customer_Contact', 'Transaction_Location', 'Transaction_ID', 'Customer_Name', 'Customer_ID']
data.drop(columns=columns_to_drop, inplace=True)

In [25]:
# View dataset
data.head()

,Gender,Age,Account_Type,Transaction_Amount,Account_Balance,Is_Fraud,Customer_Email,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_Branch_Hazaribagh Branch,Bank_Branch_Hisar Branch,Bank_Branch_Howrah Branch,Bank_Branch_Hubli Branch,Bank_Branch_Hyderabad Branch,Bank_Branch_Imphal Branch,Bank_B

### Handling Imbalance

In [26]:
# Check how manay fraudulant transaction in the dataset
data['Is_Fraud'].value_counts()

Is_Fraud
0    189912
1     10088
Name: count, dtype: int64

In [27]:
# Visualize data in Is_Fraud column
#sns.countplot(data['Is_Fraud'])

In [28]:
# Separating the data for analysis
legit = data[data.Is_Fraud == 0]
fraud = data[data.Is_Fraud == 1]

In [29]:
# Print the shape of separated data
print(legit.shape)
print(fraud.shape)

(189912, 425)
(10088, 425)


In [30]:
# Statistical measures for legit data
legit.Transaction_Amount.describe()

count    189912.000000
mean          0.000484
std           0.999653
min          -1.734662
25%          -0.863705
50%          -0.000721
75%           0.868206
max           1.732359
Name: Transaction_Amount, dtype: float64

In [31]:
# Statistical measures for fraud data
fraud.Transaction_Amount.describe()

count    10088.000000
mean        -0.009109
std          1.006575
min         -1.734658
25%         -0.879733
50%         -0.013136
75%          0.859165
max          1.732325
Name: Transaction_Amount, dtype: float64

In [32]:
# Compare the values for both transaction
data.groupby('Is_Fraud').mean()

,Gender,Age,Account_Type,Transaction_Amount,Account_Balance,Customer_Email,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Dadra and Nagar Haveli and Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Manipur,State_Meghalaya,State_Mizoram,State_Nagaland,State_Odisha,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,City_Agra,City_Ahmedabad,City_Aizawl,City_Ajmer,City_Ambala,City_Ambassa,City_Amritsar,City_Asansol,City_Aurangabad,City_Bangalore,City_Belgaum,City_Berhampur,City_Bhagalpur,City_Bhavnagar,City_Bhopal,City_Bhubaneswar,City_Bilaspur,City_Bokaro,City_Car Nicobar,City_Champhai,City_Chandigarh,City_Chennai,City_Churachandpur,City_Coimbatore,City_Cuttack,City_Daman,City_Dehradun,City_Dhanbad,City_Dharmanagar,City_Dibrugarh,City_Diglipur,City_Dimapur,City_Diu,City_Durg,City_Durgapur,City_East Delhi,City_Faridabad,City_Gangtok,City_Gaya,City_Guntur,City_Gurugram,City_Guwahati,City_Gwalior,City_Haldwani,City_Haridwar,City_Hazaribagh,City_Hisar,City_Howrah,City_Hubli,City_Hyderabad,City_Imphal,City_Indore,City_Itanagar,City_Jabalpur,City_Jagdalpur,City_Jaipur,City_Jalandhar,City_Jamshedpur,City_Jodhpur,City_Jorethang,City_Jorhat,City_Jowai,City_Kangpokpi,City_Kangra,City_Kanpur,City_Karaikal,City_Karimnagar,City_Kavaratti,City_Khammam,City_Kochi,City_Kohima,City_Kolasib,City_Kolkata,City_Korba,City_Kota,City_Kottayam,City_Kozhikode,City_Kullu,City_Lucknow,City_Ludhiana,City_Lunglei,City_Madurai,City_Mahe,City_Manali,City_Mangalore,City_Mangan,City_Mapusa,City_Margao,City_Meerut,City_Mokokchung,City_Mumbai,City_Munger,City_Muzaffarpur,City_Mysore,City_Nagaon,City_Nagpur,City_Naharlagun,City_Nainital,City_Namchi,City_Nashik,City_Nellore,City_New Delhi,City_Nizamabad,City_Nongstoin,City_North Delhi,City_Panaji,City_Patiala,City_Patna,City_Port Blair,City_Puducherry,City_Pune,City_Raipur,City_Rajkot,City_Ranchi,City_Rishikesh,City_Rourkela,City_Salem,City_Sambalpur,City_Shillong,City_Shimla,City_Silchar,City_Siliguri,City_Silvassa,City_South Delhi,City_Surat,City_Tawang,City_Thiruvananthapuram,City_Thoubal,City_Tirupati,City_Trichur,City_Trichy,City_Tura,City_Udaipur,City_Ujjain,City_Vadodara,City_Varanasi,City_Vasco da Gama,City_Vijayawada,City_Visakhapatnam,City_Warangal,City_West Delhi,City_Wokha,City_Yanam,City_Ziro,Bank_Branch_Agra Branch,Bank_Branch_Ahmedabad Branch,Bank_Branch_Aizawl Branch,Bank_Branch_Ajmer Branch,Bank_Branch_Ambala Branch,Bank_Branch_Ambassa Branch,Bank_Branch_Amritsar Branch,Bank_Branch_Asansol Branch,Bank_Branch_Aurangabad Branch,Bank_Branch_Bangalore Branch,Bank_Branch_Belgaum Branch,Bank_Branch_Berhampur Branch,Bank_Branch_Bhagalpur Branch,Bank_Branch_Bhavnagar Branch,Bank_Branch_Bhopal Branch,Bank_Branch_Bhubaneswar Branch,Bank_Branch_Bilaspur Branch,Bank_Branch_Bokaro Branch,Bank_Branch_Car Nicobar Branch,Bank_Branch_Champhai Branch,Bank_Branch_Chandigarh Branch,Bank_Branch_Chennai Branch,Bank_Branch_Churachandpur Branch,Bank_Branch_Coimbatore Branch,Bank_Branch_Cuttack Branch,Bank_Branch_Daman Branch,Bank_Branch_Dehradun Branch,Bank_Branch_Dhanbad Branch,Bank_Branch_Dharmanagar Branch,Bank_Branch_Dibrugarh Branch,Bank_Branch_Diglipur Branch,Bank_Branch_Dimapur Branch,Bank_Branch_Diu Branch,Bank_Branch_Durg Branch,Bank_Branch_Durgapur Branch,Bank_Branch_East Delhi Branch,Bank_Branch_Faridabad Branch,Bank_Branch_Gangtok Branch,Bank_Branch_Gaya Branch,Bank_Branch_Guntur Branch,Bank_Branch_Gurugram Branch,Bank_Branch_Guwahati Branch,Bank_Branch_Gwalior Branch,Bank_Branch_Haldwani Branch,Bank_Branch_Haridwar Branch,Bank_Branch_Hazaribagh Branch,Bank_Branch_Hisar Branch,Bank_Branch_Howrah Branch,Bank_Branch_Hubli Branch,Bank_Branch_Hyderabad Branch,Bank_Branch_Imphal Branch,Bank_Branch_Ind

### Over-Sampling

In [33]:
# Extract features and target from the ORIGINAL dataset (data)
X = data.drop(columns=['Is_Fraud'], axis=1)
y = data['Is_Fraud']

# Apply SMOTE to balance the dataset
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [45]:
# Split the resampled dataset into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

In [46]:
# Initialize the Logistic Regression model
log_reg = LogisticRegression()

# Train (fit) the model on the training data
log_reg.fit(X_train, y_train)


LogisticRegression()

In [47]:
# Predict fraud (0 = Legit, 1 = Fraud)
y_pred = log_reg.predict(X_test)  

# Get probability scores for fraud detection
y_prob = log_reg.predict_proba(X_test)[:, 1]  

In [48]:
# Print Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Print Classification Report (Precision, Recall, F1-score)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Print ROC-AUC Score
print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))


Confusion Matrix:
 [[36245  1738]
 [ 3691 34291]]
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93     37983
           1       0.95      0.90      0.93     37982

    accuracy                           0.93     75965
   macro avg       0.93      0.93      0.93     75965
weighted avg       0.93      0.93      0.93     75965

ROC-AUC Score: 0.9636074134321304


### Random Forest

In [49]:
# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate Random Forest Performance
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest ROC-AUC Score:", roc_auc_score(y_test, y_prob_rf))

Random Forest Confusion Matrix:
 [[37884    99]
 [ 1967 36015]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97     37983
           1       1.00      0.95      0.97     37982

    accuracy                           0.97     75965
   macro avg       0.97      0.97      0.97     75965
weighted avg       0.97      0.97      0.97     75965

Random Forest ROC-AUC Score: 0.9928491482377542
